In [1]:
import numpy as np
from lsst.sims.photUtils import PhotometricParameters
from lsst.sims.photUtils import Bandpass, Sed, SignalToNoise
import os
from lsst.utils import getPackageDir
import lsst.syseng.throughputs as st


In [2]:
def coadd_m5(m5s):
    result = 1.25*np.log10(np.sum(10.**(0.8*m5s)))
    return result

In [3]:
defaultDirs = st.setDefaultDirs()
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = st.buildHardwareAndSystem(defaultDirs, addLosses=True, 
                                             atmosphereOverride=atmos, shiftFilters=None)


In [33]:
def check_snr_m5(exptime=30, nexp=1):
    """See what the u-band m5 and snr of u=20 star is
    """
    readnoise = 0 #8.8
    effarea=np.pi*(6.423/2*100)**2
    X=1.0
    othernoise=0
    darkcurrent=0.2
    photParams_std = PhotometricParameters(exptime=exptime, nexp=nexp,
                                               gain=1.0, effarea=effarea, readnoise=readnoise,
                                               othernoise=othernoise, darkcurrent=darkcurrent)
    m5info = st.makeM5(hardware, system, exptime=exptime, nexp=nexp,
                  readnoise=readnoise, othernoise=othernoise, darkcurrent=darkcurrent,
                   effarea=effarea,X=X )
    m=20.
    snr1 = SignalToNoise.calcSNR_m5(m, system['u'], m5info['m5'][0], photParams_std)
    return snr1[0], m5info['m5'][0]

In [34]:
snr, m5 = check_snr_m5(exptime=30, nexp=1)
print('snr,  m5')
print(snr, m5)

snr,  m5
119.363105747 24.3522553135


In [35]:
snr, m5 =check_snr_m5(exptime=15, nexp=2)
print(snr, m5)

119.363105747 24.3522553135


In [36]:
snr, m5 = check_snr_m5(exptime=15, nexp=1)
# Combine two single 15s exposures. 
print(snr*np.sqrt(2), coadd_m5(np.array([m5,m5])))

119.363105747 24.3335439086


In [37]:
snr_1, m5_1 = check_snr_m5(exptime=2, nexp=1)
snr_2, m5_2 = check_snr_m5(exptime=28, nexp=1)

print(np.sqrt(snr_1**2+snr_2**2), coadd_m5(np.array([m5_1,m5_2])))

119.363105747 24.3430336579


In [9]:
## So, it looks like the SNR is always behaving in the usual way where I can add them in quadrature. 
#But, the m5 values are only consistent if makeM5 does them--I don't seem to know how to combine them properly? 


In [10]:
# What happens if I coadd 10 identical frames?
coadd_m5(np.array(10*[22.]))

23.25

In [12]:
SignalToNoise.calcSNR_m5(20, system['u'],22., None, gamma=0)

(12.559432157547898, 0)

In [13]:
SignalToNoise.calcSNR_m5(20, system['u'],23.25, None, gamma=0)

(22.334179607548151, 0)

In [14]:
SignalToNoise.calcSNR_m5(20, system['u'],22., None, gamma=0)[0]*np.sqrt(10)

39.71641173621407

In [15]:
coadd_m5(np.array(2*[22.]))

22.376287494579977

In [16]:
snr, m5 = check_snr_m5(exptime=15, nexp=1)
m5

23.957256414054967

In [20]:
snr, m5 =check_snr_m5(exptime=15, nexp=1)
print(snr, m5)
print(snr*np.sqrt(10.), m5+1.25)

84.402461497 23.9572564141
266.904018455 25.2072564141


In [21]:
snr, m5 =check_snr_m5(exptime=15, nexp=10)
print(snr, m5)

266.904018455 25.250958978
